In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/월간 11 운동')

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Oct  6 15:11:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from scipy.stats import skew
from scipy.stats import kurtosis
from sklearn.cluster import KMeans
import math
from tqdm import tqdm

train = pd.read_csv('train_features.csv')
train_acc, train_gy  = train.iloc[:, 2:5], train.iloc[:, 5:] #iloc 알아두기. 데이터에 특정 행과 열을 선택.

train_label = pd.read_csv('train_labels.csv')
train_y = train_label.label

test = pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

In [7]:
# Nsamples =600
# EulerSaved = np.zeros([Nsamples,3])
# EulerSaved.shape

**GPU 확인**

In [8]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# EDA


# DATA Augment

In [9]:
import scipy
!pip install transforms3d
from transforms3d.axangles import axangle2mat

def rolling(data):
    for j in np.random.choice(data.shape[0], int(data.shape[0]*2/3)):
        data[j] = np.roll(data[j], np.random.choice(data.shape[1]), axis= 0)
    return data

def rotation(data):
    axis = np.random.uniform(low=-1, high=1, size=data.shape[1])
    angle = np.random.uniform(low=-np.pi, high=np.pi)
    return np.matmul(data , axangle2mat(axis,angle))

def permutation(data, nPerm=4, mSL=10):
    data_new = np.zeros(data.shape)
    idx = np.random.permutation(nPerm)
    bWhile = True
    while bWhile == True:
        segs = np.zeros(nPerm+1, dtype=int)
        segs[1:-1] = np.sort(np.random.randint(mSL, data.shape[0]-mSL, nPerm-1))
        segs[-1] = data.shape[0]
        if np.min(segs[1:]-segs[0:-1]) > mSL:
            bWhile = False
    pp = 0
    for ii in range(nPerm):
        data_temp = data[segs[idx[ii]]:segs[idx[ii]+1],:]
        data_new[pp:pp+len(data_temp),:] = data_temp
        pp += len(data_temp)
    return(data_new)

"""
sigma = 0.05
def Jitter(data, sigma=0.05):
    myNoise = np.random.normal(loc=0, scale=sigma, size=data.shape)
    return data+myNoise
"""

def combine_aug(data, k, aug_P = 0):
    data_ = data.copy()
    if aug_P == 0:
        if (k+1) % 2 == 0:
            for i in np.random.choice(int(data.shape[0]/600), int(data.shape[0]/600*2/3)):
                data_[600*i:600*(i+1)] = rotation(np.array(data_[600*i:600*(i+1)]))
        if (k+1) % 2 == 1:
            for i in np.random.choice(int(data.shape[0]/600), int(data.shape[0]/600*2/3)):
                data_[600*i:600*(i+1)] = permutation(np.array(data_[600*i:600*(i+1)]))
                
    if aug_P != 0:
        pass
    return data_


     |████████████████████████████████| 62 kB 1.0 MB/s 
  Created wheel for transforms3d: filename=transforms3d-0.3.1-py3-none-any.whl size=59373 sha256=551f2c7b213db31c98dc39facdea775e8b0eca4152b946c50c2706fec337e069
  Stored in directory: /root/.cache/pip/wheels/b5/b7/93/8985551f83720ce37548a5b543c75380bb707955a9c2c5d28c
Successfully built transforms3d


# 주어진 dataset를 모델에 넣을 데이터로 변환


#에너지 변환 및 Roll & Pitch

In [10]:
# pi = math.pi
# dt=0.02 
# from math import atan, sqrt

# def roll_pitch(data):
#     roll = (data.iloc[:,1]/(data.iloc[:,0]**2 + data.iloc[:,2]**2).apply(lambda x : sqrt(x))).apply(lambda x : atan(x))*180/np.pi
#     pitch = (data.iloc[:,0]/(data.iloc[:,1]**2 + data.iloc[:,2]**2).apply(lambda x : sqrt(x))).apply(lambda x : atan(x))*180/np.pi
#     return pd.concat([roll,pitch], axis=1)

# # 원본데이터를 에너지값으로 표현
# def get_energy(data):
#     energy_ = (data.iloc[:,0]**2+data.iloc[:,1]**2+data.iloc[:,2]**2)**(1/3)
#     return energy_ 

# 칼만필터

In [11]:
    !pip install -q tensorflow-gpu==2.0.0-rc1
    !pip install transformations
    import tensorflow as tf
    import numpy as np
    from numpy.linalg import inv
    import matplotlib.pyplot as plt
    from math import cos, sin, asin, atan2, pi, atan, sqrt
    from scipy import io
    from transformations import euler_from_quaternion


     |████████████████████████████████| 380.5 MB 9.1 kB/s 
     |████████████████████████████████| 501 kB 59.9 MB/s 
     |████████████████████████████████| 50 kB 5.6 MB/s 
     |████████████████████████████████| 4.3 MB 36.7 MB/s 
     |████████████████████████████████| 45 kB 1.6 MB/s 
  Created wheel for transformations: filename=transformations-2021.6.6-cp37-cp37m-linux_x86_64.whl size=113507 sha256=82c5a3ca7018f5dd53a7dc6fcb66e7de94a3f7732bffe8ec637c04b4c4db099c
  Stored in directory: /root/.cache/pip/wheels/f0/3c/16/a67f60f017d0003f27b7156162df6f3aa45e4b2cee82232e3a
Successfully built transformations


In [26]:
 '''
 Filename: 11_EulerKalman.py
 Created on: April,3, 2021
 Author: dhpark
'''
def kalman_excute(train_acc,train_gy):

    def GetGyro(i):
        p = train_gy.iloc[i,0]  # (41500, 1)
        q = train_gy.iloc[i,1]  # (41500, 1)
        r = train_gy.iloc[i,2]  # (41500, 1)
        
        return p, q, r

    def GetAccel(i):
        ax = train_acc.iloc[i,0]   # (41500, 1)
        ay = train_acc.iloc[i,1]  # (41500, 1)
        az = train_acc.iloc[i,2] 
        
        return ax, ay, az
        
    def EulerAccel(ax, ay, az):
        theta = atan(ax/sqrt(ay**2 + az**2))
        phi = atan(ay/az)
        return phi, theta

    def EulerToQuaternion(phi, theta, psi):
        sinPhi = sin(phi/2)
        cosPhi = cos(phi/2)
        sinTheta = sin(theta/2)
        cosTheta = cos(theta/2)
        sinPsi = sin(psi/2)
        cosPsi = cos(psi/2)
        z = np.array([cosPhi*cosTheta*cosPsi + sinPhi*sinTheta*sinPsi,
                      sinPhi*cosTheta*cosPsi - cosPhi*sinTheta*sinPsi,
                      cosPhi*sinTheta*cosPsi + sinPhi*cosTheta*sinPsi,
                      cosPhi*cosTheta*sinPsi - sinPhi*sinTheta*cosPsi])
        return z

    def EulerKalman(A, z, k, Q, H, R, x, P):
        
        Xp = A @ x # Xp : State Variable Prediction
        Pp = A @ P @ A.T + Q # Error Covariance Prediction

        K = (Pp @ H.T) @ inv(H@Pp@H.T + R) # K : Kalman Gain

        x = Xp + K@(z - H@Xp) # Update State Variable Estimation
        P = Pp - K@H@Pp # Update Error Covariance Estimation
       

        phi, theta, psi = euler_from_quaternion(x)
            
        # phi   = atan2(2 * (x[2] * x[3] + x[0] * x[1]), 1 - 2*(x[1]**2 + x[2]**2))
        # try:
        #     theta = -asin(2 *  (x[1] * x[3] - x[0] * x[2]))
        # except:
        #     theta = -asin()
        # psi   = atan2(2 *  (x[1] * x[2] + x[0] * x[3]), 1-2*(x[2]**2 + x[3]**2))
        return phi, theta, psi

    Nsamples = train_acc.shape[0]
    EulerSaved = np.zeros([Nsamples,3])
    dt = 0.01

    H, Q, R = None, None, None  
    x, P = None, None

    for k in range(Nsamples):
        p, q, r = GetGyro(k)
        A = np.eye(4) + dt * (1/2) * np.array([[0,-p,-q,-r],[p,0,r,-q],[q,-r,0,p],[r,q,-p,0]])
        ax, ay, az = GetAccel(k)
        phi, theta = EulerAccel(ax, ay, az)
        z = EulerToQuaternion(phi, theta, 0) #State variable as Quaternion form
        
        H = np.eye(4)
        Q = 0.0001 * np.eye(4)
        R = 10 * np.eye(4)
        x = np.array([1, 0, 0, 0]).transpose()
        P = np.eye(4)
        phi, theta, psi = EulerKalman(A, z, k, Q, H, R, x, P)
        EulerSaved[k] = [phi, theta, psi]
       

    PhiSaved = EulerSaved[:,0] * 180/pi
    ThetaSaved = EulerSaved[:,1] * 180/pi
    PsiSaved = EulerSaved[:,2] * 180/pi


    PhiSaved = pd.DataFrame(PhiSaved)
    ThetaSaved = pd.DataFrame(ThetaSaved)
    
    return pd.concat([PhiSaved,ThetaSaved], axis=1)


'''
plt.figure()
plt.plot(t, PhiSaved)
plt.xlabel('Time [Sec]')
plt.ylabel('Roll angle [deg]')
plt.savefig('11_EulerKalman_roll.png')

plt.figure()
plt.plot(t, ThetaSaved)
plt.xlabel('Time [Sec]')
plt.ylabel('Pitch angle [deg]')
plt.savefig('11_EulerKalman_pitch.png')
plt.show()
'''

"\nplt.figure()\nplt.plot(t, PhiSaved)\nplt.xlabel('Time [Sec]')\nplt.ylabel('Roll angle [deg]')\nplt.savefig('11_EulerKalman_roll.png')\n\nplt.figure()\nplt.plot(t, ThetaSaved)\nplt.xlabel('Time [Sec]')\nplt.ylabel('Pitch angle [deg]')\nplt.savefig('11_EulerKalman_pitch.png')\nplt.show()\n"

# 변환한 데이터를 DataFrame화

In [13]:
def train_dataset(acc_data, gy_data):

    kalam_data = kalman_excute(acc_data, gy_data)

    return pd.concat([acc_data, gy_data, kalam_data], axis= 1)

def test_dataset(acc_data, gy_data):
    
    kalam_data = kalman_excute(acc_data, gy_data)

    return pd.concat([acc_data, gy_data, kalam_data], axis= 1)

# Scale Data


#Scaler fit 및 Dataset 리턴값 확인하기.

In [14]:
     '''
      import sklearn
      from sklearn.preprocessing import StandardScaler
      scaler = StandardScaler()

      train_reshape_acc = np.array(train_acc).reshape(-1, 600, 3)
      train_reshape_gy = np.array(train_gy).reshape(-1, 600, 3)

      for i in range(1000):
        train_ACC, train_GY = train_reshape_acc[i].reshape(-1, 3), train_reshape_gy[i].reshape(-1, 3)
        data_for_scaler = train_dataset(pd.DataFrame(train_ACC), pd.DataFrame(train_GY))
        print(data_for_scaler)
        scaler = StandardScaler().fit(data_for_scaler) #스탠다드 스케일러 사용

        scale_X = np.array(data_for_scaler).reshape(-1, 600, data_for_scaler.shape[1])
      #print(scale_X)
      '''

'\n import sklearn\n from sklearn.preprocessing import StandardScaler\n scaler = StandardScaler()\n\n train_reshape_acc = np.array(train_acc).reshape(-1, 600, 3)\n train_reshape_gy = np.array(train_gy).reshape(-1, 600, 3)\n\n for i in range(1000):\n   train_ACC, train_GY = train_reshape_acc[i].reshape(-1, 3), train_reshape_gy[i].reshape(-1, 3)\n   data_for_scaler = train_dataset(pd.DataFrame(train_ACC), pd.DataFrame(train_GY))\n   print(data_for_scaler)\n   scaler = StandardScaler().fit(data_for_scaler) #스탠다드 스케일러 사용\n\n   scale_X = np.array(data_for_scaler).reshape(-1, 600, data_for_scaler.shape[1])\n #print(scale_X)\n '

In [15]:
# from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
# from sklearn.pipeline import make_pipeline
# scaler = StandardScaler()

# Xtrain_scaled = pd.DataFrame(data=scaler.fit_transform(Xtrain[total_feature_names]), index=Xtrain.index, columns=total_feature_names)
# Xtest_scaled = pd.DataFrame(data=scaler.transform(Xtest[total_feature_names]), index=Xtest.index, columns=total_feature_names)

# print(Xtrain_scaled.shape, Xtest_scaled.shape)

Reshape

In [16]:
# Xtrain_scaled = np.array(Xtrain_scaled[total_feature_names]).reshape(-1, 600, len(total_feature_names)).astype('float32')
# Xtest_scaled= np.array(Xtest_scaled[total_feature_names]).reshape(-1, 600, len(total_feature_names)).astype('float32')

# Ytrain = Ytrain['label']

# print(Xtrain_scaled.shape, Ytrain.shape, Xtest_scaled.shape, submission.shape)

#Modeling

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as L
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [18]:
'''
#1D - CNN

import tensorflow as tf
import tensorflow.keras as keras
!pip install tensorflow-addons
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.layers import Dense, LSTM,Bidirectional,Dropout
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras import backend as K 
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed
import keras
'''

'\n#1D - CNN\n\nimport tensorflow as tf\nimport tensorflow.keras as keras\n!pip install tensorflow-addons\nimport tensorflow_addons as tfa\nfrom keras.models import Sequential\nfrom keras.layers import Dense, LSTM,Bidirectional,Dropout\nfrom keras.layers import Dense\nfrom keras.layers import Dropout\nfrom keras.layers import LSTM\nfrom keras.layers.convolutional import Conv1D\nfrom keras.layers.convolutional import MaxPooling1D\nfrom keras.utils import to_categorical\nfrom keras import backend as K \nfrom keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau\nfrom sklearn.model_selection import KFold,StratifiedKFold\nfrom numpy.random import seed\nimport keras\n'

# Model definition


In [19]:
'''
def cnn_model(input_shape, classes):
    seed(2021)
    tf.random.set_seed(2021)
    
    input_layer = keras.layers.Input(input_shape)
    conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)
    conv1 = keras.layers.Dropout(rate=0.3)(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    conv2 = keras.layers.Dropout(rate=0.4)(conv2)
    
    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    conv3 = keras.layers.Dropout(rate=0.5)(conv3)
    
    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    
    output_layer = keras.layers.Dense(classes, activation='softmax')(gap)
    
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), 
        metrics=['accuracy'])
    
    return model
'''

"\ndef cnn_model(input_shape, classes):\n    seed(2021)\n    tf.random.set_seed(2021)\n    \n    input_layer = keras.layers.Input(input_shape)\n    conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)\n    conv1 = keras.layers.BatchNormalization()(conv1)\n    conv1 = keras.layers.Activation(activation='relu')(conv1)\n    conv1 = keras.layers.Dropout(rate=0.3)(conv1)\n\n    conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)\n    conv2 = keras.layers.BatchNormalization()(conv2)\n    conv2 = keras.layers.Activation('relu')(conv2)\n    conv2 = keras.layers.Dropout(rate=0.4)(conv2)\n    \n    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)\n    conv3 = keras.layers.BatchNormalization()(conv3)\n    conv3 = keras.layers.Activation('relu')(conv3)\n    conv3 = keras.layers.Dropout(rate=0.5)(conv3)\n    \n    gap = keras.layers.GlobalAveragePooling1D()(conv3)\n    \n    output_layer = keras.layers.Dense(classes, ac

In [20]:
def first_model(input) :

    
    inputs = L.Input(shape = (input.shape[1], input.shape[2]))
    gru1 = L.GRU(256, return_sequences = True, dropout = 0.2)(inputs)
    mp = L.MaxPool1D()(gru1)
    ap = L.AveragePooling1D()(gru1)
    concat1 = L.Concatenate()([mp, ap])
    gru2 = L.GRU(256, return_sequences = True, dropout = 0.2)(concat1)
    GAP = L.GlobalAveragePooling1D()(gru2)
    dense = L.Dense(61, activation = "softmax")(GAP)
    return keras.models.Model(inputs, dense)

In [21]:
"""
train_split_id = [1,2,5,7]
train_reshape_acc[train_split_id].shape
"""

'\ntrain_split_id = [1,2,5,7]\ntrain_reshape_acc[train_split_id].shape\n'

In [22]:
# i = list(range(10))
# print(i)

In [23]:
# b = [1,2,3]
# my_2darray = np.array([[1, 2, 3], [4, 5, 6],[6,74,7],[7,4,3],[7,86,3]])
# a= pd.DataFrame(my_2darray)
# print(c)
# c=np.array(a)
# print(c[b])
# x = pd.DataFrame(np.array(a)[b])
# print(x)

In [24]:
    # train_reshape_acc = np.array(train_acc).reshape(-1, 600, 3)
    # train_reshape_gy = np.array(train_gy).reshape(-1, 600, 3)
    
    # #print(train_reshape_acc, "train reshape \n")
    

    # #i = list(range(0, 3125))
    # i = [1,2,3]
    # k=[1,2]
    # train_ACC, train_GY = train_reshape_acc[i].reshape(-1, 3), train_reshape_gy[i].reshape(-1, 3)
    # x =[]
   
    # print(train_ACC)
    # for id in k :
    #   a = 600*(id-1)
    #   b = 600*id
    #   x.append(train_ACC[a:b])
    # y = np.array(x)
    # x1 = y.reshape(y.shape[0]*y.shape[1],y.shape[2])  
    # pd.DataFrame(x1)
    # print("x1: \n",x1
    #       )
    # print("x1: \n",x1.shape)
    

# Train Model 

In [ ]:
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import accuracy_score
    import random
    import sklearn
    from sklearn.preprocessing import StandardScaler
     

    # first_rlr : 첫번째로 learning_rate이 감소
    # second_rlr : 두번째로 learning_rate이 감소
    # r_seed : StratifiedKFold seed
    #eed_ : numpy/random seed
    
    epochs = 40
    first_rlr = 15
    second_rlr = 30
    r_seed = 2020
    seed_ = 25

    train_reshape_acc = np.array(train_acc).reshape(-1, 600, 3)
    train_reshape_gy = np.array(train_gy).reshape(-1, 600, 3)

    scaler = StandardScaler()

    i = list(range(0, 3125))
    #i = list(range(0, 100))
    train_ACC, train_GY = train_reshape_acc[i].reshape(-1, 3), train_reshape_gy[i].reshape(-1, 3)
    data_for_scaler = train_dataset(pd.DataFrame(train_ACC), pd.DataFrame(train_GY))
    scale_X = scaler.fit_transform(data_for_scaler) #스탠다드 스케일러 사용

    scale_X = np.array(scale_X).reshape(-1, 600, data_for_scaler.shape[1])
    # train_scaler_ACC, train_scaler_GY = train_reshape_acc.reshape(-1, 3), train_reshape_gy.reshape(-1, 3)
    # data_for_scaler = train_dataset(pd.DataFrame(train_scaler_ACC), pd.DataFrame(train_scaler_GY))
    
    # scaler = StandardScaler().fit(np.array(data_for_scaler)) #스탠다드 스케일러 사용
    # data_for_scaler = np.array(data_for_scaler).reshape(-1, 600, data_for_scaler.shape[1])

    result_model = []
    cnt = 0
    
    random.seed(seed_)
    tf.random.set_seed(21)

    print(scale_X.shape[1],"Scale" ,scale_X.shape[2])
    model = first_model(scale_X)
    model.compile(optimizer=keras.optimizers.RMSprop(0.003),
                  loss='categorical_crossentropy', metrics=['accuracy'])

    spliter = StratifiedKFold(n_splits=10, shuffle = True, random_state = r_seed)

    for train_split_id, valid_split_id in spliter.split(train_reshape_acc, train_y):
        
        train_Y, valid_Y = np.array(pd.get_dummies(train_y))[train_split_id], np.array(pd.get_dummies(train_y))[valid_split_id]
        valid_ACC, valid_GY = train_reshape_acc[valid_split_id].reshape(-1, 3), train_reshape_gy[valid_split_id].reshape(-1, 3)
        valid_x = test_dataset(pd.DataFrame(valid_ACC), pd.DataFrame(valid_GY))
        print("valid_x","\n",valid_x,"\n")
        valid_X = scaler.transform(np.array(valid_x)).reshape(-1, 600, valid_x.shape[1])
 
       
        val_score = 0
        seed_ += 1

        for i in range(epochs):
            
            np.random.seed(seed_*47 + i)

            # print("train_split_id",train_split_id,"\n")
            # print("valid_split_id",valid_split_id,"\n")
            # train_ACC, train_GY = train_reshape_acc[train_split_id].reshape(-1, 3), train_reshape_gy[train_split_id].reshape(-1, 3)

            # # print("train_ACC",train_ACC,"\n")
            # # print("train_GY",train_GY,"\n")
            # train_x = train_dataset(pd.DataFrame(train_ACC), pd.DataFrame(train_GY))
            # print("train_x","\n",train_x,"\n")
 
            train_x = []
            # print(train_split_id)
            # print(data_for_scaler)
            
            a, b = train_id_slice(train_split_id, data_for_scaler)    
            train_x.extend(np.array(data_for_scaler)[a:b])
            
            train_x_arr = np.array(train_x)
            # pd.DataFrame(train_x_arr)
            # print("train_x_arr","\n",train_x_arr,"\n")
            # print("train_x_arr","\n",train_x_arr.shape,"\n")
            # train_x_arr1 = train_x_arr.reshape(-1,3) 
            # print("train_x_arr1","\n",train_x_arr1,"\n")
            train_X = scaler.transform(train_x_arr).reshape(-1, 600, pd.DataFrame(train_x_arr).shape[1])
        
            train_X_ = train_X.copy()

            train_X_ = rolling(train_X_)
            # print("train_X_",train_X_,"\n", "train_X:" ,train_X,"\n")
            hist = model.fit(train_X_, train_Y, epochs = 1, validation_data = (valid_X, valid_Y), verbose = 0)

            train_accuracy = hist.history["accuracy"]
            new_val_score = accuracy_score(np.argmax(valid_Y, axis = 1), np.argmax(model.predict(valid_X), axis = 1))
            val_loss = hist.history["val_loss"]

            if i == first_rlr:
                model.compile(optimizer=keras.optimizers.RMSprop(0.003*0.2),
                              loss='categorical_crossentropy', metrics=['accuracy'])

            if i == second_rlr:
                model.compile(optimizer = keras.optimizers.RMSprop(0.003*0.2*0.4),
                             loss='categorical_crossentropy', metrics=['accuracy'])

            print("epoch {} - train_accuracy : {} - validation_loss : {} - validation_accuracy : {}".format(i,
                                                                                                            train_accuracy,
                                                                                                            val_loss,
                                                                                                            new_val_score,
                                                                                                            ))

            if i == 0:
                val_loss_score = val_loss[0]
        
            if val_loss_score >= val_loss[0]:
                val_loss_score = val_loss[0]
                best_model = model
                print("####best_val####")
                    
            if new_val_score >= val_score:
                val_score = new_val_score
                best_model = model
                print("####best_acc####")
        print("####################################################### cycle {} is done".format(cnt))
        result_model.append(best_model)
        cnt+=1

    def train_id_slice(train_split_id, data_for_scaler):
              for id in train_split_id:
              
                a = 600*id
                b = 600*(id+1)

                yield a, b

    def predict_(model):
        result = []
        for mod in model:
            result.append(mod.predict(test_X))
        predict = np.array(result).mean(axis = 0)

        return predict

    def save_model(models, name = '1'):
        cnt = 1
        for model in models:
            model.save(path + "submission/last/weight/" + name + '-{}.h5'.format(cnt))
            cnt +=1



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in double_scalars


In [ ]:
# model = first_model(scale_X)
# print(model.history)

#시작

제출

In [ ]:
submission.to_csv('kalman_submission.csv', index=False)